<a href="https://colab.research.google.com/github/rosanth/QnAA/blob/main/QNAA_FINISHED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**

1.   [Distilbert](https://colab.research.google.com/drive/1HDZl5sb4DwFXvpLOzhtWchP-kZ1NPzyx#scrollTo=XPr8yftHc3Ar&line=1&uniqifier=1)
2.   [Vilt processor](https://colab.research.google.com/drive/1gUmjdzs5d04VZ9VtLFXHThfrKggCF3bj#scrollTo=FPz7ZK_NdXHI)
1.   [Tapas](https://colab.research.google.com/drive/1mjFDYQ0NgQEVg78cb6tl_qdPykk4i_hh#scrollTo=oF-RDEMsdsNw)
2.   Bart
1.   Blip processor










In [ ]:
!pip install PyMuPDF

In [ ]:
import ipywidgets as widgets # create interface
from IPython.display import display, clear_output, HTML
from PIL import Image
from transformers import (
    BartForConditionalGeneration, BartTokenizer,
    BlipProcessor, BlipForConditionalGeneration,
    pipeline,
    ViltProcessor, ViltForQuestionAnswering
)
import fitz  # PyMuPDF
import io # used to file handling
import os # read and write the file
import time
import pandas as pd
from ipywidgets import widgets, Accordion, Layout, AppLayout

In [ ]:
# Load pre-trained distilbert model  for PDF QA
qa_pipeline_pdf = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Load pre-trained VILT processor and model for Image
processor_image = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model_image = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

# Load pre-trained tapas model  for csv QA
tqa = pipeline(task="table-question-answering", model="google/tapas-large-finetuned-wtq")

# Load pre-trained BART model and tokenizer for PDF summarization
pdf_model_name = "facebook/bart-large-cnn"
pdf_tokenizer = BartTokenizer.from_pretrained(pdf_model_name)
pdf_model = BartForConditionalGeneration.from_pretrained(pdf_model_name)

# Load the Blip processor and model for image summarization
image_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
image_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")




Device set to use cpu
Device set to use cpu


In [ ]:


# Widgets for file upload and page navigation
file_chooser_pdf = widgets.FileUpload(
    description="Upload",
    accept=".pdf",
    multiple=False,
    style={'button_color': '#a3a8a5'},
    layout=widgets.Layout(width='400px')
)

page_slider = widgets.IntSlider(value=1, min=1, max=1, step=1, description='Page:')

# Output widgets
output = widgets.Output()
output_pdf = widgets.Output()

# Chat area to display questions and answers
chat_area = widgets.VBox([])

# Question entry widget
question_entry_pdf = widgets.Text(
    placeholder="Enter your question...",
    layout=widgets.Layout(width='300px', height='50px')
)

# Function to render PDF page
def render_page(page_number):
    page = pdf.load_page(page_number - 1)  # page numbers are 0-based in PyMuPDF
    pix = page.get_pixmap()
    img = Image.open(io.BytesIO(pix.tobytes("png")))
    return img

def on_page_change(change):
    with output:
        output.clear_output(wait=True)
        display(render_page(change['new']))

page_slider.observe(on_page_change, names='value')

def on_file_upload(change):
    try:
        global pdf, num_pages
        uploaded_file = next(iter(file_chooser_pdf.value.values()))
        file_content = uploaded_file['content']

        # Add Horizontal rule
        Horizontal_rule = widgets.HTML(
            value="""
            <hr style="
                border: none;
                border-top: 2px solid #ddd;
                margin-top: 20px;
                margin-bottom: 20px;
                width: 100%;
                height: 2px;
                background-color: #ddd;">
            """
        )
        chat_area.children = list(chat_area.children) + [Horizontal_rule]

        # Load the PDF from the uploaded file content
        pdf = fitz.open(stream=file_content, filetype='pdf')

        # Get the total number of pages
        num_pages = pdf.page_count

        # Update the page slider's maximum value
        page_slider.max = num_pages

        # Render the first page initially
        with output:
            output.clear_output(wait=True)
            display(render_page(1))
        page_slider.value = 1
    except Exception as e:
        with output:
            output.clear_output()
            error_message = widgets.HTML(
                f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">Error loading PDF</span>
                        </div>
                    </div>"""
            )
            chat_area.children = list(chat_area.children) + [error_message]

file_chooser_pdf.observe(on_file_upload, names='value')

def extract_text_from_pdf(pdf_content):
    doc = fitz.open("pdf", pdf_content)
    text = ""
    for page_num in range(len(doc)):
        page = doc[page_num]
        text += page.get_text()
    doc.close()  # Close the document
    return text

def handle_errors():
    errors = []
    if not file_chooser_pdf.value:
        errors.append("Error: Please upload a PDF file.")
    if not question_entry_pdf.value:
        errors.append("Error: Please enter a question.")

    if errors:
        chat_area.children = []  # Clear previous errors
        for error in errors:
            error_message = widgets.HTML(
                f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">{error}</span>
                        </div>
                    </div>"""
            )
            chat_area.children = list(chat_area.children) + [error_message]
        return True
    return False

def generate_button_click_pdf(b):
    if handle_errors():
        return

    try:
        pdf_file_content = next(iter(file_chooser_pdf.value.values()))["content"]
        pdf_text = extract_text_from_pdf(pdf_file_content)
        question = question_entry_pdf.value

        with output_pdf:
            output_pdf.clear_output()
            # Assuming you have a qa_pipeline_pdf function to get the answer
            answer = qa_pipeline_pdf(question=question, context=pdf_text)

            user_message_html = f"""
            <div style="display: flex; justify-content: flex-end; margin-top: 5px;">
                <div style="background-color: #ddf; padding: 10px; border-radius: 10px; max-width: 60%;">
                    {question}
                </div>
            </div>
            """

            response_message_html = f"""
            <div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                <div style="background-color: #fdd; padding: 10px; border-radius: 10px; max-width: 60%;">
                    Answer: {answer['answer']}
                </div>
            </div>
            """

            chat_area.children = list(chat_area.children) + [
                widgets.HTML(user_message_html),
                widgets.HTML(response_message_html)
            ]

        question_entry_pdf.value = ''  # Clear the question input box
    except Exception as e:
        with output_pdf:
            output_pdf.clear_output()
            error_message = widgets.HTML(f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">Error loading PDF</span>
                        </div>
                    </div>""")
            chat_area.children = list(chat_area.children) + [error_message]

with output:
    display(widgets.HTML("Please upload a PDF file."))

# Generate Button for PDF
generate_button_pdf = widgets.Button(description="Get Answer", style={'button_color': '#a3a8a5'})
generate_button_pdf.on_click(generate_button_click_pdf)




In [ ]:


# Create widgets for Image
file_chooser_image = widgets.FileUpload(accept='image/*', multiple=False, description="Upload", style={'button_color': '#a3a8a5'}, layout=widgets.Layout(width='400px'))
image_widget = widgets.Image(layout={'width': '300px', 'height': '300px'})
question_entry_image = widgets.Text(placeholder='Enter your question...', layout=widgets.Layout(width='350px', height='50px'))
generate_button_image = widgets.Button(description='Get Answer', style={'button_color': '#a3a8a5'})
output_widget_image = widgets.Output()
chat_area_img = widgets.VBox([])
output_image = widgets.Output()

# Define image variable for Image
image = None

# Function to handle file selection for Image
def on_file_upload_image(change):
    global image
    output_widget_image.clear_output()  # Clear old output when a new image is uploaded
    try:
        if file_chooser_image.value:
            # Get the uploaded image
            image_data = next(iter(file_chooser_image.value.values()))
            image = Image.open(io.BytesIO(image_data['content']))

            # Add Horizontal rule
            Horizontal_rule = widgets.HTML(
                value="""
                <hr style="
                    border: none;
                    border-top: 2px solid #ddd;
                    margin-top: 20px;
                    margin-bottom: 20px;
                    width: 100%;
                    height: 2px;
                    background-color: #ddd;">
                """
            )
            chat_area_img.children = list(chat_area_img.children) + [Horizontal_rule]

            # Resize the image to a specific size (e.g., 300x300)
            image = image.resize((300, 300))

            # Display the image
            with output_widget_image:
                display(image_widget)
                image_widget.value = image_data['content']
    except Exception as e:
        error_message = widgets.HTML(f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">Error loading image</span>
                        </div>
                    </div>""")
        chat_area_img.children = list(chat_area_img.children) + [error_message]

file_chooser_image.observe(on_file_upload_image, names='_counter')

# Function to handle errors
def handle_errors_image():
    errors = []
    if not file_chooser_image.value:
        errors.append("Error: Please upload an image file.")
    if not question_entry_image.value:
        errors.append("Error: Please enter a question.")

    if errors:
        with output_image:
            output_image.clear_output()
            for error in errors:
                error_message = widgets.HTML( f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">{error}</span>
                        </div>
                    </div>""")
                chat_area_img.children = list(chat_area_img.children) + [error_message]
        return True
    return False

# Function to handle button click for Image
def generate_answer_image(button):
    if handle_errors_image():
        return

    try:
        with output_image:
            output_image.clear_output()  # Clear old output when a new question is asked
            if image:
                # Get the question
                text = question_entry_image.value

                # Prepare inputs
                encoding = processor_image(image, text, return_tensors="pt")

                # Forward pass
                outputs = model_image(**encoding)
                logits = outputs.logits
                idx = logits.argmax(-1).item()
                predicted_answer = model_image.config.id2label[idx]

                # Update chat interface
                user_message_img_html = f"""
                    <div style="display: flex; justify-content: flex-end; margin-top: 5px;">
                        <div style="background-color: #ddf; padding: 10px; border-radius: 10px; max-width: 60%;">
                            {text}
                        </div>
                    </div>
                """

                response_message_img_html = f"""
                    <div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #fdd; padding: 10px; border-radius: 10px; max-width: 60%;">
                            Answer: {predicted_answer}
                        </div>
                    </div>
                """

                chat_area_img.children = list(chat_area_img.children) + [
                    widgets.HTML(user_message_img_html),
                    widgets.HTML(response_message_img_html)
                ]

                question_entry_image.value = ''  # Clear the question input box
    except Exception as e:
        error_message = widgets.HTML(f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">Error loading Image</span>
                        </div>
                    </div>""")
        chat_area_img.children = list(chat_area_img.children) + [error_message]

# Attach the function to the button click event for Image
generate_button_image.on_click(generate_answer_image)

# Display the widgets
#display(file_chooser_image, image_widget, output_widget_image, question_entry_image, generate_button_image, output_image, chat_area_img)


In [ ]:


# Initialize global variables
df = pd.DataFrame()
num_rows = 0

# Create chat area for displaying questions and answers
chat_area_csv = widgets.VBox([])

# Function to handle button click event
def on_button_click(b):
    query = question_entry_csv.value
    answer_output_csv.clear_output()

def handle_errors_csv():
    errors = []
    if not file_upload_csv.value:
        errors.append("Error: Please upload a CSV file.")
    if not question_entry_csv.value:
        errors.append("Error: Please enter a question.")

    if errors:
        with answer_output_csv:
            answer_output_csv.clear_output()
            for error in errors:
                error_message = widgets.HTML( f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">{error}</span>
                        </div>
                    </div>""")
                chat_area_csv.children = list(chat_area_csv.children) + [error_message]
        return True
    return False

def generate_answer_csv(button):
    if handle_errors_csv():
        return

    try:
        with answer_output_csv:
            answer_output_csv.clear_output()  # Clear old output when a new question is asked
            if file_upload_csv.value:
                # Get the uploaded file content
                uploaded_file = list(file_upload_csv.value.values())[0]['content']
                table = pd.read_csv(io.BytesIO(uploaded_file), encoding='utf-8')
                table = table.astype(str)

                # Get the query
                query = question_entry_csv.value

                # Get the answer from the TQA model (assuming a function tqa exists)
                answer = tqa(table=table, query=query)["answer"]

                if 'count' in answer.lower():
                    split = answer.split(',')
                    count = len(split)
                    result = str(count)
                elif "average" in answer.lower():
                    # Extract the numeric part of the answer
                    numeric_part = answer.split(">")[-1].strip()
                    result = "Answer: " + numeric_part
                else:
                    result = answer

                # Update chat interface
                user_message_csv_html = f"""
                    <div style="display: flex; justify-content: flex-end; margin-top: 5px;">
                        <div style="background-color: #ddf; padding: 10px; border-radius: 10px; max-width: 60%;">
                            {query}
                        </div>
                    </div>
                """

                response_message_csv_html = f"""
                    <div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #fdd; padding: 10px; border-radius: 10px; max-width: 60%;">
                            Answer: {result}
                        </div>
                    </div>
                """

                chat_area_csv.children = list(chat_area_csv.children) + [
                    widgets.HTML(user_message_csv_html),
                    widgets.HTML(response_message_csv_html)
                ]

                question_entry_csv.value = ''  # Clear the question input box
    except Exception as e:
        error_message = widgets.HTML(f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">Error loading CSV</span>
                        </div>
                    </div>""")
        chat_area_csv.children = list(chat_area_csv.children) + [error_message]

# File uploader for CSV
file_upload_csv = widgets.FileUpload(accept=".csv", multiple=False, description="Upload", style={'button_color': '#a3a8a5'}, layout=widgets.Layout(width='400px'))

# Text input for question for CSV
question_entry_csv = widgets.Text(placeholder="Enter your question...", layout=widgets.Layout(width='350px', height='50px'))

# Button to trigger the question answering process for CSV
answer_button_csv = widgets.Button(description="Get Answer", style={'button_color': '#a3a8a5'})
answer_button_csv.on_click(generate_answer_csv)

# Viewer components
page_csv_slider = widgets.IntSlider(value=1, min=1, max=1, step=1, description='Page:')
rows_per_csv_page = widgets.IntText(value=10, description='Rows per page:', style={'description_width': 'initial'})
output_csv = widgets.Output()

# Define a function to display a page of the DataFrame
def display_page(page_number, rows_per_page):
    start_row = (page_number - 1) * rows_per_page
    end_row = start_row + rows_per_page
    return df.iloc[start_row:end_row]

def on_file_upload(change):
    global df, num_rows
    try:
        uploaded_filename = next(iter(file_upload_csv.value))
        content = file_upload_csv.value[uploaded_filename]['content']
        df = pd.read_csv(io.BytesIO(content))
        num_rows = len(df)
        page_csv_slider.max = (num_rows + rows_per_csv_page.value - 1) // rows_per_csv_page.value
        page_csv_slider.value = 1

        # Add Horizontal rule
        Horizontal_rule = widgets.HTML(
                value="""
                <hr style="
                    border: none;
                    border-top: 2px solid #ddd;
                    margin-top: 20px;
                    margin-bottom: 20px;
                    width: 100%;
                    height: 2px;
                    background-color: #ddd;">
                """
            )
        chat_area_csv.children = list(chat_area_csv.children) + [Horizontal_rule]

        with output_csv:
            output_csv.clear_output(wait=True)
            display(display_page(1, rows_per_csv_page.value))
    except Exception as e:
        error_message = widgets.HTML(f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">Error loading CSV</span>
                        </div>
                    </div>""")
        chat_area_csv.children = list(chat_area_csv.children) + [error_message]

def on_page_change(change):
    with output_csv:
        output_csv.clear_output(wait=True)
        display(display_page(page_csv_slider.value, rows_per_csv_page.value))

def on_rows_per_page_change(change):
    with output_csv:
        output_csv.clear_output(wait=True)
        display(display_page(page_csv_slider.value, rows_per_csv_page.value))
    page_csv_slider.max = (num_rows + rows_per_csv_page.value - 1) // rows_per_csv_page.value

# Set up observers
file_upload_csv.observe(on_file_upload, names='value')
page_csv_slider.observe(on_page_change, names='value')
rows_per_csv_page.observe(on_rows_per_page_change, names='value')

# Output widget to display the answer for CSV
answer_output_csv = widgets.Output()

# Display the widgets
#display(file_upload_csv, question_entry_csv, answer_button_csv, answer_output_csv, page_csv_slider, rows_per_csv_page, output_csv, chat_area_csv)


In [ ]:


# Define widgets for PDF summarization
pdf_file_chooser = widgets.FileUpload(
    accept=".pdf",
    multiple=False,
    description="Upload",
    style={'button_color': '#a3a8a5'},
    layout=widgets.Layout(width='400px')
)
pdf_generate_button = widgets.Button(description="Generate Summary", style={'button_color': '#a3a8a5'}, layout=widgets.Layout(width='400px'))
pdf_output_text_widget = widgets.Output()
outputpdf = widgets.Output()
page_pdf_slider = widgets.IntSlider(value=1, min=1, max=1, step=1, description='Page:')

# Loading message widget
loading_message = widgets.HTML(
    "<div style='display: flex; justify-content: center; align-items: center;'>"
    "<span style='font-size: 16px; color: #555;'>Generating summary, please wait...</span>"
    "</div>"
)

# Function to render page
def render_page(page_number):
    page = pdf.load_page(page_number - 1)  # page numbers are 0-based in PyMuPDF
    pix = page.get_pixmap()
    img = Image.open(io.BytesIO(pix.tobytes("png")))
    return img

def on_page_change(change):
    with outputpdf:
        outputpdf.clear_output(wait=True)
        display(render_page(change['new']))

page_pdf_slider.observe(on_page_change, names='value')

def on_file_upload(change):
    global pdf, num_pages
    try:
        uploaded_file = next(iter(pdf_file_chooser.value.values()))
        file_content = uploaded_file['content']

        # Load the PDF from the uploaded file content
        pdf = fitz.open(stream=file_content, filetype='pdf')

        # Get the total number of pages
        num_pages = pdf.page_count

        # Update the page slider's maximum value
        page_pdf_slider.max = num_pages

        # Render the first page initially
        with outputpdf:
            outputpdf.clear_output(wait=True)
            display(render_page(1))
        page_pdf_slider.value = 1
    except Exception as e:
        with pdf_output_text_widget:
            outputpdf.clear_output()
            error_message = f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">Invalid File Format</span>
                        </div>
                    </div>"""
            display(widgets.HTML(error_message))

pdf_file_chooser.observe(on_file_upload, names='value')

# Define function to be called when the PDF summary button is clicked
def on_pdf_generate_button_click(b):
    with pdf_output_text_widget:
        pdf_output_text_widget.clear_output()


        # Check if a PDF file is uploaded
        if not pdf_file_chooser.value:
            error_html = f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">Please Choose PDF file</span>
                        </div>
                    </div>"""
            display(widgets.HTML(error_html))
            return


        # Get the uploaded PDF file content
        pdf_content = next(iter(pdf_file_chooser.value.values()))

        try:
            display(loading_message)
            # Measure time taken for summarization
            start_time = time.time()

            with fitz.Document(stream=pdf_content["content"], filetype="pdf") as pdf_doc:
                # Extract text from all pages
                pdf_text = ""
                for page_num in range(pdf_doc.page_count):
                    page = pdf_doc[page_num]
                    pdf_text += page.get_text()

            input_ids = pdf_tokenizer.encode(pdf_text, return_tensors="pt", max_length=1024, truncation=True)

            # Generate summary
            summary_ids = pdf_model.generate(input_ids, max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
            summary = pdf_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Print the generated PDF summary
            summary_html = f"""
                <div style="display: flex; flex-direction: column; margin-top: 5px;">
                    <div style="background-color: #ddf; padding: 10px; border-radius: 10px; max-width: 100%;">
                        <strong>Generated PDF Summary:</strong>
                        <p>{summary}</p>
                    </div>
                    <div style="background-color: #86f7ad; padding: 10px; border-radius: 10px; max-width: 100%;">
                        Time taken for PDF summarization: {time.time() - start_time:.2f} seconds
                    </div>
                </div>
            """
            display(widgets.HTML(summary_html))

        except Exception as e:

            error_html = f"""<div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                        <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                            <span style="color: white;">Please select PDF</span>
                        </div>
                    </div>"""
            display(widgets.HTML(error_html))
        finally:
            loading_message.close()

# Attach the function to the PDF summary button's click event
pdf_generate_button.on_click(on_pdf_generate_button_click)




In [ ]:
# Widgets for image summarization
image_file_uploader = widgets.FileUpload(
    accept="image/*",
    multiple=False,
    description="Upload",
    style={'button_color': '#a3a8a5'},
    layout=widgets.Layout(width='400px')
)
image_generate_button = widgets.Button(description="Generate Summary", style={'button_color': '#a3a8a5'}, layout=widgets.Layout(width='400px'))
image_output_text_widget = widgets.Output()
image_suz_widget = widgets.Image(layout={'width': '300px', 'height': '300px'})
output_suz_image = widgets.Output()



# Function to generate captions based on input text for image summarization
def generate_image_caption(text, raw_image):
    inputs = image_processor(raw_image, text, return_tensors="pt")
    out = image_model.generate(**inputs)
    return image_processor.decode(out[0], skip_special_tokens=True)

# Define image variable for Image
raw_image = None

# Function to handle file selection for image summarization
def on_image_file_upload(change):
    global raw_image

    with output_suz_image:
        try:
            # Clear the previous outputs
            output_suz_image.clear_output()
            image_output_text_widget.clear_output()

            # Check if a file is uploaded
            if not image_file_uploader.value:
                error_html = """
                <div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                    <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                        <span style="color: white;">Please choose an image file.</span>
                    </div>
                </div>"""
                display(widgets.HTML(error_html))
                return

            # Get the uploaded file content
            uploaded_file = image_file_uploader.value[next(iter(image_file_uploader.value))]

            # Read the content of the file
            file_content = uploaded_file["content"]

            # Check if the file is an image
            image = Image.open(io.BytesIO(file_content))
            if image.format not in ["JPEG", "PNG", "JPG", "GIF"]:
                raise ValueError("Invalid file format.")

            # Create a PIL Image from the file content
            raw_image = image.convert('RGB')
            raw_image = raw_image.resize((300, 300))

            # Set the value of image_suz_widget
            image_suz_widget.value = file_content


        except Exception as e:
            # Display error message
            error_html = """
                <div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                    <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                        <span style="color: white;">Invalid File Format</span>
                    </div>
                </div>"""
            display(widgets.HTML(error_html))

def generate_summary(change):
    with image_output_text_widget, output_suz_image:
        try:
            # Check if raw_image is set
            if raw_image is None:
                error_html = """
                <div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                    <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                        <span style="color: white;">Please choose an image file.</span>
                    </div>
                </div>"""
                display(widgets.HTML(error_html))
                return

            # Clear previous outputs and display loading message
            output_suz_image.clear_output()
            image_output_text_widget.clear_output()
            display(loading_message)

            # Generate captions
            start_time = time.time()
            title = generate_image_caption("", raw_image)
            description = generate_image_caption("This is a picture of", raw_image)
            colors = generate_image_caption("The colors are", raw_image)
            objects = generate_image_caption("There are", raw_image)
            end_time = time.time()

            # Display captions
            captions_html = f"""
                <div style="display: flex; flex-direction: column; margin-top: 5px;">
                    <div style="background-color: #ddf; padding: 10px; border-radius: 10px; max-width: 100%; margin-bottom: 5px;">
                        <strong>Title:</strong> {title}<br>
                        <strong>Description:</strong> {description}<br>
                        <strong>Colors:</strong> {colors}<br>
                        <strong>Objects:</strong> {objects}
                    </div>
                    <div style="background-color: #86f7ad; padding: 10px; border-radius: 10px; max-width: 100%;">
                        Time taken for image processing: {end_time - start_time:.2f} seconds
                    </div>
                </div>
            """
            image_output_text_widget.clear_output()  # Clear the loading message
            display(widgets.HTML(captions_html))

        except Exception as e:
            # Display error message
            error_html = """
                <div style="display: flex; justify-content: flex-start; margin-top: 5px;">
                    <div style="background-color: #f50505; padding: 10px; border-radius: 10px; max-width: 100%;">
                        <span style="color: white;">Error: {str(e)}</span>
                    </div>
                </div>"""
            image_output_text_widget.clear_output()  # Clear the loading message
            display(widgets.HTML(error_html))

# Set up the event handler for image file upload
image_file_uploader.observe(on_image_file_upload, names="value")

# Attach the function to the image summary button's click event
image_generate_button.on_click(generate_summary)

# Display widgets
#display(image_file_uploader, image_generate_button, image_output_text_widget, output_suz_image,image_suz_widget)

In [ ]:

chatcsv = AppLayout(
    header=file_upload_csv,
    left_sidebar= None,  # Arrange Accordions vertically in the left sidebar
    center= chat_area_csv,  # Display Accordions in the center
    right_sidebar=None,
    footer=widgets.HBox([question_entry_csv,answer_button_csv]),
    pane_heights=[1, '325px', 1]

)
chatimg = AppLayout(
    header=file_chooser_image,
    left_sidebar= None,  # Arrange Accordions vertically in the left sidebar
    center= chat_area_img,  # Display Accordions in the center
    right_sidebar=None,
    footer=widgets.HBox([question_entry_image,generate_button_image]),
    pane_heights=[1, '325px', 1]

)

chatpdf = AppLayout(
    header=file_chooser_pdf,
    left_sidebar= None,  # Arrange Accordions vertically in the left sidebar
    center= chat_area,  # Display Accordions in the center
    right_sidebar=None,
    footer=widgets.HBox([question_entry_pdf,generate_button_pdf]),
    pane_heights=[1, '325px', 1]

)

appdf = AppLayout(
    header=None,
    left_sidebar=None,  # Arrange Accordions vertically in the left sidebar
    center=widgets.VBox([chatpdf]),  # Display Accordions in the center
    right_sidebar=widgets.VBox([page_slider,output]),
    footer=None,
    pane_widths=[0,2,2]
)

apcsv = AppLayout(
    header=None,
    left_sidebar=None,  # Arrange Accordions vertically in the left sidebar
    center= widgets.VBox([chatcsv]),  # Display Accordions in the center
    right_sidebar=widgets.VBox([page_csv_slider,rows_per_csv_page,output_csv]),
    footer= None,
    pane_widths=[0,2,2]
)
apimg = AppLayout(
    header=None,
    left_sidebar=None,  # Arrange Accordions vertically in the left sidebar
    center=widgets.VBox([chatimg]),  # Display Accordions in the center
    right_sidebar=widgets.VBox([output_widget_image]) ,
    footer= None,
    pane_widths=[0,2,2]
)
chatsupdf = AppLayout(
    header=pdf_file_chooser,
    left_sidebar= None,  # Arrange Accordions vertically in the left sidebar
    center= pdf_output_text_widget,  # Display Accordions in the center
    right_sidebar=None,
    footer=pdf_generate_button,
    pane_heights=[1, '325px', 1]

)
supdf = AppLayout(
    header=None,
    left_sidebar=None,  # Arrange Accordions vertically in the left sidebar
    center=widgets.VBox([chatsupdf]),  # Display Accordions in the center
    right_sidebar=widgets.VBox([page_pdf_slider,outputpdf]) ,
    footer= None,
    pane_widths=[0,2,2]
)

chatsuimg = AppLayout(
    header=image_file_uploader,
    left_sidebar= None,  # Arrange Accordions vertically in the left sidebar
    center= output_suz_image,  # Display Accordions in the center
    right_sidebar=None,
    footer=image_generate_button,
    pane_heights=[1, '325px', 1]

)
suimg = AppLayout(
    header=None,
    left_sidebar=None,  # Arrange Accordions vertically in the left sidebar
    center=widgets.VBox([chatsuimg]),  # Display Accordions in the center
    right_sidebar=image_suz_widget ,
    footer= None,
    pane_widths=[0,2,2]
)
layout_image = widgets.VBox([apimg],layout=Layout(height='400px', width='100%'))
layout_pdf = widgets.VBox([appdf],layout=Layout(height='400px', width='100%'))

layout_csv = widgets.VBox([apcsv ],layout=Layout(height='400px', width='100%'))
pdf_frame = widgets.VBox([supdf ], layout=Layout(height='400px', width='100%'))
image_frame = widgets.VBox([suimg ], layout=Layout(height='400px', width='100%'))


In [ ]:
from ipywidgets import widgets, Layout, Accordion, AppLayout

# Create the styled header widget with rounded corners
header = widgets.HTML("""
    <h2 style='
        color: white;
        background-color: #6200EA;
        text-align: center;
        padding: 10px;
        border-top-left-radius: 15px;
        border-top-right-radius: 15px;
    '>
        QnA-A <br> AI powered
    </h2>
""")
footer = widgets.HTML("""
    <h2 style='
        color: white;
        background-color: #6200EA;
        text-align: center;
        padding: 10px;
        border-bottom-left-radius: 15px;
        border-bottom-right-radius: 15px;
    '>
        Question and Answering - Assistant
    </h2>
""")


intro_html1 = """
<div style='
    display: inline-block;
    position: relative;
    background: #f0f0f0;
    border: 2px solid blue;
    border-radius: 10px;
    padding: 20px;
    margin: 20px;
    font-size: 15px;
    font-weight: bold;
    color: blue;
    text-align: left;
'>
    Hi there! I'm QnA-A. I can process your PDFs, images, and CSV files. <br>
    Ask me questions or let me summarize your documents using advanced machine learning models

    <div style='
        content: "";
        position: absolute;
        border-style: solid;
        border-width: 15px 15px 15px 0;
        border-color: transparent blue transparent transparent;
        display: block;
        width: 0;
        z-index: 1;
        left: -15px;
        top: 20px;
    '></div>
</div>
"""

intro_html2 = """
<div style='
    display: inline-block;
    position: relative;
    background: #f0f0f0;
    border: 2px solid blue;
    border-radius: 10px;
    padding: 20px;
    margin: 20px;
    font-size: 15px;
    font-weight: bold;
    color: blue;
    text-align: left;
'>

    Let's chat with your files!!
    <div style='
        content: "";
        position: absolute;
        border-style: solid;
        border-width: 15px 15px 15px 0;
        border-color: transparent blue transparent transparent;
        display: block;
        width: 0;
        z-index: 1;
        left: -15px;
        top: 20px;
    '></div>
</div>
"""

intro_img1 ="""<img src="https://static.wixstatic.com/media/cbbfa4_6837841839e8454ca66998053fd8bfa7~mv2.gif" height="375px" width="250px">"""
# Create section_home with styled label
aphome = AppLayout(
    header=None,
    left_sidebar=widgets.HTML(value=intro_img1),  # Arrange Accordions vertically in the left sidebar
    center=widgets.VBox([widgets.HTML(value=intro_html1),widgets.HTML(value=intro_html2)]),  # Display Accordions in the center
    right_sidebar=None ,
    footer= None
)
section_home = widgets.VBox([aphome], layout=Layout(height='400px', width='100%'))


# Create the Accordion widgets and set titles
accordion_home = Accordion(children=[section_home])
accordion_home.set_title(0, 'Home')
accordion1 = Accordion(children=[layout_pdf])
accordion1.set_title(0, 'PDF Chat')
accordion2 = Accordion(children=[layout_csv])
accordion2.set_title(0, 'CSV Chat')
accordion3 = Accordion(children=[layout_image])
accordion3.set_title(0, 'Image Chat')
accordion4 = Accordion(children=[pdf_frame])
accordion4.set_title(0, 'PDF Summary')
accordion5 = Accordion(children=[image_frame])
accordion5.set_title(0, 'Image Summary')

# Initially hide the Accordions
accordion_home.layout.display = 'block'
accordion1.layout.display = 'none'
accordion2.layout.display = 'none'
accordion3.layout.display = 'none'
accordion4.layout.display = 'none'
accordion5.layout.display = 'none'

# Create the buttons to show/hide the Accordions
toggle_button_home = widgets.Button(description ="Home",layout=Layout( width='265px'),style=dict(text_color='red'))
toggle_button1 = widgets.Button(description="PDF",layout=Layout( width='265px'))
toggle_button2 = widgets.Button(description="CSV",layout=Layout( width='265px'))
toggle_button3 = widgets.Button(description="Image",layout=Layout( width='265px'))
toggle_button4 = widgets.Button(description="PDF",layout=Layout( width='265px'))
toggle_button5 = widgets.Button(description="Image",layout=Layout( width='265px'))

# Define the functions to toggle Accordion visibility
def toggle_accordion(b, index):
    accordions = [accordion_home, accordion1, accordion2, accordion3, accordion4, accordion5]
    for i, accordion in enumerate(accordions):
        accordion.layout.display = 'block' if i == index else 'none'

# Connect the button click events to the functions
toggle_button_home.on_click(lambda b: toggle_accordion(b, 0))
toggle_button1.on_click(lambda b: toggle_accordion(b, 1))
toggle_button2.on_click(lambda b: toggle_accordion(b, 2))
toggle_button3.on_click(lambda b: toggle_accordion(b, 3))
toggle_button4.on_click(lambda b: toggle_accordion(b, 4))
toggle_button5.on_click(lambda b: toggle_accordion(b, 5))

# Combine the buttons in a VBox for each Accordion
homeset = widgets.VBox([toggle_button_home], layout=Layout(height='50px', width='100%'))
QAset = widgets.VBox([toggle_button1, toggle_button2, toggle_button3], layout=Layout(height='115px', width='100%'))
Suzset = widgets.VBox([toggle_button4, toggle_button5], layout=Layout(height='85px', width='100%'))
QA = Accordion(children=[QAset])
QA.set_title(0, 'Question And Answering')
Suz = Accordion(children=[Suzset])
Suz.set_title(0, 'Summarizing')
Home = Accordion(children=[homeset])
Home.set_title(0, 'Home')

#  Combine Accordion widgets in a VBox
accordion_box = widgets.VBox([accordion_home,accordion1, accordion2, accordion3, accordion4, accordion5])

#  Make the VBox scrollable
accordion_box_scrollable = widgets.VBox([accordion_box], layout=Layout(overflow='scroll', height='500px'))

#  Create the AppLayout
app = AppLayout(
    header=header,
    left_sidebar=widgets.VBox([Home, QA, Suz]),  # Arrange Accordions vertically in the left sidebar
    center=accordion_box_scrollable,  # Display Accordions in the center
    right_sidebar=None,
    footer=footer
)

#  Display the AppLayout
display(app)


AppLayout(children=(HTML(value="\n    <h2 style='\n        color: white;\n        background-color: #6200EA;\n…